In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2021-09-10 02:58:09,3.0707,172.7902,82.76,74,6,13.35
1,1,Ushuaia,AR,2021-09-10 02:55:13,-54.8000,-68.3000,47.86,46,40,11.50
2,2,Rikitea,PF,2021-09-10 02:59:57,-23.1203,-134.9692,71.62,86,73,19.15
3,3,Pereslavl-Zalesskiy,RU,2021-09-10 02:59:57,56.7393,38.8563,55.44,90,85,9.86
4,4,Mount Gambier,AU,2021-09-10 02:59:57,-37.8333,140.7667,58.91,63,100,12.66


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Create DF of cities that fall between the customer's desired temperatures
preferred_cities_df  = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2021-09-10 02:58:09,3.0707,172.7902,82.76,74,6,13.35
5,5,Hasaki,JP,2021-09-10 02:59:57,35.7333,140.8333,82.80,74,9,11.14
7,7,Surt,LY,2021-09-10 02:59:58,31.2089,16.5887,76.08,61,0,12.62
9,9,Atuona,PF,2021-09-10 02:55:32,-9.8000,-139.0333,78.60,75,3,10.76
10,10,Manokwari,ID,2021-09-10 02:59:58,-0.8667,134.0833,84.11,70,100,10.74
...,...,...,...,...,...,...,...,...,...,...
530,530,Nhulunbuy,AU,2021-09-10 03:02:50,-12.2333,136.7667,86.61,54,75,20.71
533,533,Esna,EG,2021-09-10 03:02:51,25.2934,32.5540,83.46,43,31,19.57
537,537,Deming,US,2021-09-10 02:59:34,32.2687,-107.7586,80.24,33,0,1.36
539,539,Naze,JP,2021-09-10 03:02:53,28.3667,129.4833,82.74,76,16,8.32


In [12]:
preferred_cities_df.count()

City_ID       187
City          187
Country       187
Date          187
Lat           187
Lng           187
Max Temp      187
Humidity      187
Cloudiness    187
Wind Speed    187
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Butaritari,KI,82.76,3.0707,172.7902,
5,Hasaki,JP,82.80,35.7333,140.8333,
7,Surt,LY,76.08,31.2089,16.5887,
9,Atuona,PF,78.60,-9.8000,-139.0333,
10,Manokwari,ID,84.11,-0.8667,134.0833,
11,Pontianak,ID,85.98,-0.0333,109.3333,
15,Kendari,ID,78.04,-3.9450,122.4989,
20,Matara,LK,82.20,5.9485,80.5353,
23,Randudongkal,ID,87.15,-7.0981,109.3243,
26,Qui Nhon,VN,81.95,13.7667,109.2333,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Butaritari,KI,82.76,3.0707,172.7902,Isles Sunset Lodge
5,Hasaki,JP,82.80,35.7333,140.8333,Hotel Sunrise Choshi
7,Surt,LY,76.08,31.2089,16.5887,الهادي ابوشعاله سرت
9,Atuona,PF,78.60,-9.8000,-139.0333,Villa Enata
10,Manokwari,ID,84.11,-0.8667,134.0833,Swiss Belhotel Manokwari
11,Pontianak,ID,85.98,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
15,Kendari,ID,78.04,-3.9450,122.4989,Hotel Wixel
20,Matara,LK,82.20,5.9485,80.5353,Amaloh Boutique Resort
23,Randudongkal,ID,87.15,-7.0981,109.3243,M N D P
26,Qui Nhon,VN,81.95,13.7667,109.2333,Hoang Yen Canary Hotel


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))